# Introduction to Deep Learning with PyTorch

Neste caderno, você será apresentado ao [PyTorch] (http://pytorch.org/), uma estrutura para construir e treinar redes neurais. O PyTorch se comporta de várias maneiras como as matrizes que você ama do Numpy. Afinal, essas matrizes Numpy são apenas tensores. O PyTorch pega esses tensores e facilita a sua transferência para GPUs para o processamento mais rápido necessário ao treinar redes neurais. Ele também fornece um módulo que calcula automaticamente gradientes (para retropropagação!) E outro módulo especificamente para a construção de redes neurais. No conjunto, o PyTorch acaba sendo mais coerente com o Python e a pilha Numpy / Scipy em comparação com o TensorFlow e outras estruturas.



## Neural Networks

O Deep Learning é baseado em redes neurais artificiais que existem de alguma forma desde o final da década de 1950. As redes são construídas a partir de partes individuais que se aproximam dos neurônios, normalmente chamadas de unidades ou simplesmente "neurônios". Cada unidade possui algum número de entradas ponderadas. Essas entradas ponderadas são somadas (uma combinação linear) e depois passadas por uma função de ativação para obter a saída da unidade.

<img src="assets/simple_neuron.png" width=400px>

Matematicamente, isso se parece com:

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

Com vetores, este é o produto interno / ponto de dois vetores:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensors

Acontece que os cálculos de redes neurais são apenas um monte de operações de álgebra linear em * tensores *, uma generalização de matrizes. Um vetor é um tensor unidimensional, uma matriz é um tensor bidimensional, uma matriz com três índices é um tensor tridimensional (imagens em cores RGB, por exemplo). A estrutura de dados fundamental para redes neurais são os tensores e o PyTorch (assim como praticamente qualquer outra estrutura de aprendizado profundo) é construído em torno dos tensores.

<img src="assets/tensor_examples.svg" width=600px>

Com o básico abordado, é hora de explorar como podemos usar o PyTorch para construir uma rede neural simples.

In [1]:
# First, import PyTorch
import torch

In [2]:
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [3]:
### Generate some data --Gere alguns dados
torch.manual_seed(7) # Set the random seed so things are predictable -- Defina a semente aleatória para que as coisas sejam previsíveis

# Features are 5 random normal variables -- Os recursos são 5 variáveis normais aleatórias
features = torch.randn((1, 5))
# True weights for our data, random normal variables again -- Pesos reais para nossos dados, variáveis normais aleatórias novamente
weights = torch.randn_like(features)
# and a true bias term -- e um termo verdadeiro viés
bias = torch.randn((1, 1))

Acima, geramos dados que podemos usar para obter a saída de nossa rede simples. Por enquanto, tudo é aleatório. No futuro, começaremos a usar dados normais. Passando por cada linha relevante:

`features = torch.randn ((1, 5))` cria um tensor com a forma `(1, 5)`, uma linha e cinco colunas, que contém valores distribuídos aleatoriamente de acordo com a distribuição normal, com média de zero e padrão desvio de um.

`pesos = torch.randn_like (features)` cria outro tensor com a mesma forma que `features`, novamente contendo valores de uma distribuição normal.

Finalmente, `bias = torch.randn ((1, 1))` cria um valor único a partir de uma distribuição normal.

Os tensores PyTorch podem ser adicionados, multiplicados, subtraídos, etc., assim como os arrays Numpy. Em geral, você usará os tensores PyTorch da mesma maneira que usaria as matrizes Numpy. Eles trazem alguns benefícios interessantes, como a aceleração da GPU, que veremos mais adiante. Por enquanto, use os dados gerados para calcular a saída dessa rede simples de camada única.
> ** Exercício **: Calcule a saída da rede com recursos de entrada `recursos`, pesos` pesos` e bias `bias`. Semelhante ao Numpy, o PyTorch possui a função [`torch.sum ()`] (https://pytorch.org/docs/stable/torch.html#torch.sum), além do método `.sum ()` em tensores, para somar somas. Use a função `ativação 'definida acima como a função de ativação.

In [4]:
## Calcule a saída desta rede usando os pesos e tensores de polarização
y = activation(torch.sum(features *weights) + bias)

Você pode fazer a multiplicação e somar na mesma operação usando uma multiplicação de matrizes. Em geral, convém usar multiplicações de matriz, pois elas são mais eficientes e aceleradas usando bibliotecas modernas e computação de alto desempenho em GPUs.

Aqui, queremos fazer uma multiplicação matricial dos recursos e pesos. Para isso, podemos usar [`torch.mm ()`] (https://pytorch.org/docs/stable/torch.html#torch.mm) ou [`torch.matmul ()`] (https: // pytorch.org/docs/stable/torch.html#torch.matmul), que é um pouco mais complicado e suporta transmissão. Se tentarmos fazê-lo com os recursos e pesos como eles são, obteremos um erro

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

Ao criar redes neurais em qualquer estrutura, você verá isso com frequência. Realmente frequentemente. O que está acontecendo aqui é que nossos tensores não têm as formas corretas para realizar uma multiplicação de matrizes. Lembre-se de que para multiplicações de matrizes, o número de colunas no primeiro tensor deve ser igual ao número de linhas na segunda coluna. Ambos os `recursos` e` pesos` têm a mesma forma, `(1, 5)`. Isso significa que precisamos alterar a forma dos pesos para que a multiplicação da matriz funcione.

**Nota:** Para ver a forma de um tensor chamado `tensor`, use` tensor.shape`. Se você estiver construindo redes neurais, estará usando esse método frequentemente.

Existem algumas opções aqui: [`weights.reshape ()`] (https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize _ ()`] ( https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_) e [`weights.view ()`] (https://pytorch.org/docs/stable/tensors.html# tocha.Tensor.view).

* `weights.reshape (a, b)` retornará um novo tensor com os mesmos dados que `pesos com tamanho` `(a, b)`as vezes e às vezes um clone, pois ele copia os dados para outra parte do memória.
* `weights.resize_ (a, b)` retorna o mesmo tensor com uma forma diferente. No entanto, se a nova forma resultar em menos elementos que o tensor original, alguns elementos serão removidos do tensor (mas não da memória). Se a nova forma resultar em mais elementos que o tensor original, novos elementos serão não inicializados na memória. Aqui, devo observar que o sublinhado no final do método indica que esse método é realizado ** no local **. Aqui está um ótimo tópico do fórum para [leia mais sobre operações no local] (https://discuss.pytorch.org/t/what-is-in-place-operation/16244) no PyTorch.
* `weights.view (a, b)` retornará um novo tensor com os mesmos dados que `weights` com tamanho` (a, b) `.

Eu normalmente uso `.view ()`, mas qualquer um dos três métodos funcionará para isso. Portanto, agora podemos remodelar `pesos` 'para ter cinco linhas e uma coluna com algo como` pesos.view (5, 1) `.

> **Exercício**: Calcule a saída de nossa pequena rede usando multiplicação de matrizes.

In [5]:
## Calculate the output of this network using matrix multiplication
print(features.shape)
print(weights.shape)

y = activation(torch.mm(features, weights.view(5,1)) + bias)
print(y)


torch.Size([1, 5])
torch.Size([1, 5])
tensor([[0.1595]])


### Stack them up!

É assim que você pode calcular a saída para um único neurônio. O poder real desse algoritmo acontece quando você começa a empilhar essas unidades individuais em camadas e pilhas de camadas, em uma rede de neurônios. A saída de uma camada de neurônios se torna a entrada para a próxima camada. Com várias unidades de entrada e unidades de saída, agora precisamos expressar os pesos como uma matriz.

<img src='assets/multilayer_diagram_weights.png' width=450px>

A primeira camada mostrada na parte inferior aqui são as entradas, compreensivelmente chamadas de **camada de entrada**. A camada do meio é chamada de **camada oculta** e a camada final (à direita) é a **camada de saída**. Podemos expressar essa rede matematicamente com matrizes novamente e usar a multiplicação de matrizes para obter combinações lineares para cada unidade em uma operação. Por exemplo, a camada oculta ($ h_1 $ e $ h_2 $ aqui) pode ser calculada 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

A saída para esta pequena rede é encontrada tratando a camada oculta como entradas para a unidade de saída. A saída da rede é expressa simplesmente

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [6]:
### Generate some data
torch.manual_seed(7) # Defina a semente aleatória para que as coisas sejam previsíveis

# Os recursos são três variáveis normais aleatórias
features = torch.randn((1, 3))

# Defina o tamanho de cada camada em nossa rede
n_input = features.shape[1]     # Número de unidades de entrada, deve corresponder ao número de recursos de entrada
print(n_input)
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Pesos para entradas na camada oculta
W1 = torch.randn(n_input, n_hidden)
# Pesos da camada oculta para a camada de saída
W2 = torch.randn(n_hidden, n_output)

# e termos de polarização para camadas ocultas e de saída
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

3


> **Exercise:** Calcule a saída desta rede multicamada usando os pesos `W1` e` W2` e os desvios, `B1` e` B2`. 

In [13]:
## Your solution here
h = activation(torch.mm(features, W1) + B1)
output = activation(torch.mm(h, W2) + B2)
print(output)

tensor([[0.3171]])


Se você fez isso corretamente, deverá ver a saída `tensor ([[0,3171]])`.

O número de unidades ocultas é um parâmetro da rede, geralmente chamado de **hyperparameter** para diferenciá-lo dos parâmetros de pesos e desvios. Como você verá mais adiante, quando discutirmos o treinamento de uma rede neural, quanto mais unidades ocultas uma rede tiver e mais camadas, maior será a capacidade de aprender com os dados e fazer previsões precisas.

## Numpy to Torch and back

Seção de bônus especial! O PyTorch possui um ótimo recurso para converter entre matrizes Numpy e Torch tensors. Para criar um tensor a partir de uma matriz Numpy, use `torch.from_numpy ()`. Para converter um tensor em uma matriz Numpy, use o método `.numpy ()`.

In [8]:
import numpy as np
a = np.random.rand(4,3)
a

array([[0.24067663, 0.47803715, 0.7864128 ],
       [0.42600446, 0.3888375 , 0.13350729],
       [0.74851001, 0.82425433, 0.96667231],
       [0.36474919, 0.85049331, 0.01584525]])

In [9]:
b = torch.from_numpy(a)
b

tensor([[0.2407, 0.4780, 0.7864],
        [0.4260, 0.3888, 0.1335],
        [0.7485, 0.8243, 0.9667],
        [0.3647, 0.8505, 0.0158]], dtype=torch.float64)

In [10]:
b.numpy()

array([[0.24067663, 0.47803715, 0.7864128 ],
       [0.42600446, 0.3888375 , 0.13350729],
       [0.74851001, 0.82425433, 0.96667231],
       [0.36474919, 0.85049331, 0.01584525]])

A memória é compartilhada entre a matriz Numpy e o Torch tensor, portanto, se você alterar os valores no lugar de um objeto, o outro também será alterado.

In [11]:
# Multiply PyTorch Tensor by 2, in place -- Multiplique o PyTorch Tensor por 2, no lugar
b.mul_(2)

tensor([[0.4814, 0.9561, 1.5728],
        [0.8520, 0.7777, 0.2670],
        [1.4970, 1.6485, 1.9333],
        [0.7295, 1.7010, 0.0317]], dtype=torch.float64)

In [12]:
# Numpy array matches new values from Tensor -- A matriz numpy corresponde a novos valores do tensor
a

array([[0.48135326, 0.9560743 , 1.57282561],
       [0.85200891, 0.77767499, 0.26701457],
       [1.49702002, 1.64850866, 1.93334462],
       [0.72949838, 1.70098661, 0.0316905 ]])